
# Búsqueda Aproximada de Vecinos Más Cercanos (ANN) con LanceDB

 **IVF_PQ** 
(Indexado basado en archivos invertidos y cuantización por producto) 
sirve para realizar búsquedas rápidas en grandes conjuntos de vectores. 




In [2]:

# Cargar librerías necesarias
import lancedb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [10]:

# Configurar LanceDB
uri = "data/ANN"  # Directorio donde se guardará la base de datos
db = lancedb.connect(uri)

# Crear 10,000 vectores de muestra aleatorios
np.random.seed(42)  # Para reproducibilidad
data = [
    {"vector": row, "item": f"item {i}"}
    for i, row in enumerate(np.random.random((10_000, 1536)).astype("float32"))
]

# Crear tabla en la base de datos
tbl = db.create_table("mis_vectores", data=data)
print(f"Tabla 'mis_vectores' creada con {len(data)} vectores.")


Tabla 'mis_vectores' creada con 10000 vectores.


In [ ]:

# Crear un índice para optimizar búsquedas
tbl.create_index(num_partitions=256, num_sub_vectors=96)
print("Índice IVF_PQ creado con 256 particiones y 96 sub-vectores.")


In [ ]:

# Crear un vector de consulta aleatorio
query_vector = np.random.random((1536)).astype("float32")

# Realizar búsqueda con parámetros ajustados
resultados = tbl.search(query_vector).limit(5).nprobes(20).refine_factor(10).to_pandas()

# Mostrar resultados
print("Resultados de la búsqueda:")
print(resultados)


In [ ]:

import time

# Crear un vector de consulta
query_vector = np.random.random((1536)).astype("float32")

# Búsqueda exhaustiva (sin índice)
start_time = time.time()
exhaustive_results = tbl.search(query_vector).limit(5).to_pandas()
exhaustive_time = time.time() - start_time
print(f"Búsqueda exhaustiva tomó: {exhaustive_time:.4f} segundos")

# Búsqueda ANN (con índice)
start_time = time.time()
ann_results = tbl.search(query_vector).limit(5).nprobes(20).to_pandas()
ann_time = time.time() - start_time
print(f"Búsqueda ANN tomó: {ann_time:.4f} segundos")

# Comparar tiempos
print(f"Diferencia de tiempo (exhaustiva vs ANN): {exhaustive_time - ann_time:.4f} segundos")


In [ ]:

# Filtrar resultados para excluir un ítem específico
resultados_filtrados = tbl.search(query_vector).where("item != 'item 1141'").to_pandas()
print("Resultados filtrados:")
print(resultados_filtrados)

# Seleccionar solo ciertas columnas
resultados_proyectados = tbl.search(query_vector).select(["vector", "_distance"]).to_pandas()
print("Resultados proyectados:")
print(resultados_proyectados)


In [ ]:

# Visualizar las distancias de los resultados obtenidos
plt.barh(resultados["item"], resultados["_distance"], color="skyblue")
plt.xlabel("Distancia")
plt.ylabel("Item")
plt.title("Resultados de búsqueda ANN (Distancia)")
plt.gca().invert_yaxis()  # Invertir eje Y para legibilidad
plt.show()


In [ ]:

# Crear índice usando GPU (si se cuenta con una GPU y PyTorch con soporte CUDA)
# Requiere instalar PyTorch compatible con CUDA.

# tbl.create_index(
#     num_partitions=256,
#     num_sub_vectors=96,
#     accelerator="cuda"  # Usar "cuda" para GPUs Nvidia
# )
# print("Índice creado usando GPU.")



# Conclusión

En este notebook, aprendiste a:
1. Crear una tabla en LanceDB con datos vectoriales.
2. Construir un índice **IVF_PQ** para búsquedas rápidas.
3. Configurar parámetros como `nprobes` y `refine_factor` para optimizar resultados.
4. Aplicar filtros y proyecciones para personalizar búsquedas.
5. (Opcional) Utilizar una GPU para acelerar la creación del índice.
6. Visualizar distancias de los resultados para análisis.

Este flujo es ideal para aplicaciones como sistemas de recomendación, búsqueda de imágenes o análisis de datos en grandes escalas.
